In [1]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
import time
from multiprocessing import Pool
import numpy as np
import pandas as pd
from math import sqrt
import tensorflow as tf

In [2]:
np.random.seed(42)
train_set = open("ml-1m/ratings.dat").readlines()
train_set = np.random.permutation(train_set)[:100000]
train_threshold = int(0.75 * len(train_set))
# train_indices = []
# test_indices = []
train_user_indices = []
train_item_indices = []
test_user_indices = []
test_item_indices = []
train_ratings = []
test_ratings = []

data = defaultdict(dict)
user2id = {}
item2id = {}
index_user = 0
index_item = 0

for i, line in enumerate(train_set):
    user = line.split("::")[0]
    item = line.split("::")[1]
    rating = line.split("::")[2]
    
    try:
        user_id = user2id[user]
    except KeyError:
        user_id = index_user
        user2id[user] = index_user
        index_user += 1
    try:
        item_id = item2id[item]
    except KeyError:
        item_id = index_item
        item2id[item] = index_item
        index_item += 1
        
    if i < train_threshold:
    #    train_indices.append((user_id, item_id))
        train_user_indices.append(user_id)
        train_item_indices.append(item_id)
        train_ratings.append(int(rating))
        data[user_id].update(dict(zip([item_id], [int(rating)])))
    else:
    #    test_indices.append((user_id, item_id))
        test_user_indices.append(user_id)
        test_item_indices.append(item_id)
        test_ratings.append(int(rating))

In [11]:
len(train_ratings), len(test_ratings)

(75000, 24851)

In [ ]:
print("before: ", len(train_ratings), len(test_ratings))

for u, i, r in zip(test_user_indices, test_item_indices, test_ratings):
    if u not in data:
        test_user_indices.remove(u)
        test_item_indices.remove(i)
        test_ratings.remove(r)

for u, i, r in zip(test_user_indices, test_item_indices, test_ratings):
    if u not in data:
        test_user_indices.remove(u)
        test_item_indices.remove(i)
        test_ratings.remove(r)

for u, i, r in zip(test_user_indices, test_item_indices, test_ratings):
    if u not in data:
        test_user_indices.remove(u)
        test_item_indices.remove(i)
        test_ratings.remove(r)

for u in test_user_indices:
    if u not in data:
        print("left", u)

print("after: ", len(train_ratings), len(test_ratings))

In [10]:
def ratings(dataset):
    for user, r in dataset.items():
        for item, rating in r.items():
            yield user, item, rating

### compute baseline  -bbu & bbi 

In [12]:
def compute_rmse(mode="train"):
    if mode == "train":
        user_indices = train_user_indices
        item_indices = train_item_indices
        ratings = train_ratings
    elif mode == "test":
        user_indices = test_user_indices
        item_indices = test_item_indices
        ratings = test_ratings
        
    global_mean = np.mean(train_ratings)
    pred = global_mean + bbu[user_indices] + bbi[item_indices] 
    score = np.sqrt(np.mean(np.power(pred - ratings, 2)))
    return score

In [13]:
np.random.seed(42)
n_users = len(user2id)
n_items = len(item2id)
global_mean = np.mean(train_ratings)
n_epochs = 200
lr = 0.002
reg = 0.1
bbu = np.zeros((n_users,))
bbi = np.zeros((n_items,))
for epoch in range(n_epochs):
    for u, i, r in ratings(data):
        before = bbu.copy()
        err = r - (global_mean + bbu[u] + bbi[i])
        bbu[u] += lr * (err - reg * bbu[u])
        bbi[i] += lr * (err - reg * bbi[i])
    if epoch % 5 == 0:
        print("Epoch: ", epoch + 1, "training RMSE: ", compute_rmse("train"), "test RMSE: ", compute_rmse("test"))
        if np.allclose(before, bbu, atol=1e-3):
            print("\n Converged !")
            break

Epoch:  1 
 [0. 0. 0.] [ 0.01270422  0.00449975 -0.00621123] 
 training RMSE:  1.082633895818155 test RMSE:  1.1072706366391476
Epoch:  6 
 [ 0.04795376  0.01812561 -0.02609946] [ 0.05458544  0.02079154 -0.02953371] 
 training RMSE:  0.994043442247672 test RMSE:  1.1263636412036437
Epoch:  11 
 [ 0.07648154  0.02975694 -0.0380893 ] [ 0.08106817  0.03169735 -0.03916873] 
 training RMSE:  0.9561714049221693 test RMSE:  1.1451769040451114
Epoch:  16 
 [ 0.09671783  0.0387131  -0.04045559] [ 0.10004352  0.04032698 -0.04017856] 
 training RMSE:  0.9345015756204222 test RMSE:  1.159071352031592
Epoch:  21 
 [ 0.11140197  0.0463996  -0.03741628] [ 0.11381112  0.04784068 -0.03640691] 
 training RMSE:  0.9202352338958125 test RMSE:  1.1695170496530063
Epoch:  26 
 [ 0.12201599  0.05337048 -0.03153351] [ 0.12375157  0.05470243 -0.03016219] 
 training RMSE:  0.9100149969778989 test RMSE:  1.1776539732547588
Epoch:  31 
 [ 0.12965354  0.05986203 -0.0243235 ] [ 0.13090114  0.06111383 -0.02280815] 


### neighborhood model

In [22]:
def compute_rmse_neighbor2(mode="train"):
    if mode == "train":
        user_indices = train_user_indices
        item_indices = train_item_indices
        ratings = train_ratings
    elif mode == "test":
        user_indices = test_user_indices
        item_indices = test_item_indices
        ratings = test_ratings
        
    global_mean = np.mean(train_ratings)
    score = 0
    for u, i, r in zip(user_indices, item_indices, ratings):
        ru = np.sum([(data[u][j] - (global_mean + bbu[u] + bbi[j])) * w[i][j] for j in data[u]]) / np.sqrt(len(data[u]))
        nu = np.sum(c[i][j] for j in data[u]) / np.sqrt(len(data[u]))
        pred = global_mean + bu[u] + bi[i] + ru + nu
        score += np.power(r - pred, 2)
        
    return np.sqrt(score / len(user_indices))

In [9]:
def compute_rmse_neighbor(mode="train"):
    if mode == "train":
        user_indices = train_user_indices
        item_indices = train_item_indices
        ratings = train_ratings
    elif mode == "test":
        user_indices = test_user_indices
        item_indices = test_item_indices
        ratings = test_ratings
        
    global_mean = np.mean(train_ratings)
    score = 0
    pred_whole = []
    for u, i, r in zip(user_indices, item_indices, ratings):
        u_items = list(data[u].keys())
        u_ratings = np.array(list(data[u].values()))
        base_neighbor = global_mean + bbu[u] + bbi[u_items]
        ru = np.sum((u_ratings - base_neighbor) * w[i][u_items]) / np.sqrt(len(u_items))
        nu = np.sum(c[i][u_items]) / np.sqrt(len(u_items))
        pred = global_mean + bu[u] + bi[i] + ru + nu
        pred_whole.append(pred)
    return np.sqrt(np.mean(np.power(pred_whole - np.array(ratings), 2)))

In [ ]:
np.random.seed(42)
n_factors = 100
n_users = len(user2id)
n_items = len(item2id)
global_mean = np.mean(train_ratings)
n_epochs = 200
lr = 0.002
reg = 0.1

bu = np.zeros((n_users,))
bi = np.zeros((n_items,))
w = np.random.normal(size=(n_items, n_items))
c = np.random.normal(size=(n_items, n_items))

for epoch in range(n_epochs):
    t0 = time.time()
    for u, i, r in ratings(data):
        u_items = list(data[u].keys())
        u_ratings = np.array(list(data[u].values()))
        base_neighbor = global_mean + bbu[u] + bbi[u_items]
        ru = np.sum((u_ratings - base_neighbor) * w[i][u_items]) / np.sqrt(len(u_items))
        nu = np.sum(c[i][u_items]) / np.sqrt(len(u_items))
        err = r - (global_mean + bu[u] + bi[i] + ru + nu)
        
        bu[u] += lr * (err - reg * bu[u])
        bi[i] += lr * (err - reg * bi[i])
        w[i][u_items] += lr * (err * (u_ratings - base_neighbor) / np.sqrt(len(u_items)) - reg * w[i][u_items])
        c[i][u_items] += lr * (err / np.sqrt(len(u_items)) - reg * c[i][u_items])
        
    print("Epoch: ", epoch + 1, "\ttime: ", time.time() - t0)
    
    if epoch % 1 == 0:
        t1 = time.time()
        print("Epoch: ", epoch + 1, "\ttraining RMSE: ", compute_rmse_neighbor("train"), 
              "\ttest RMSE: ", compute_rmse_neighbor("test"))
        print("evaluate time: ", time.time() - t1)

Epoch:  1 	time:  187.67700004577637
Epoch:  1 	training RMSE:  1.5707351584800056 	test RMSE:  1.5892486851216328
Epoch:  2 	time:  182.19700002670288
Epoch:  2 	training RMSE:  1.5124462521222735 	test RMSE:  1.5422257806994302
Epoch:  3 	time:  188.18200016021729
Epoch:  3 	training RMSE:  1.4717891146854394 	test RMSE:  1.5105524429685973
Epoch:  4 	time:  183.32200002670288
Epoch:  4 	training RMSE:  1.4384903137461458 	test RMSE:  1.484972240071947
Epoch:  5 	time:  183.36899995803833
Epoch:  5 	training RMSE:  1.4095409181830287 	test RMSE:  1.462886534267612
Epoch:  6 	time:  183.37600016593933
Epoch:  6 	training RMSE:  1.3836254339147265 	test RMSE:  1.4431975553409213
Epoch:  7 	time:  183.22199988365173
Epoch:  7 	training RMSE:  1.3600257865302288 	test RMSE:  1.4253223979343055
Epoch:  8 	time:  186.3880000114441
Epoch:  8 	training RMSE:  1.3382923758738394 	test RMSE:  1.4089037340413537
Epoch:  9 	time:  184.489000082016
Epoch:  9 	training RMSE:  1.3181169312252223 	t

### neighborhood model k

In [14]:
def cosine_similarity(dicts, x1, x2):  # only common ratings
    prod = denom1 = denom2 = 0
    for item in dicts[x1]:
        if item in dicts[x2]:
            prod += dicts[x1][item] * dicts[x2][item]
            denom1 += dicts[x1][item] ** 2
            denom2 += dicts[x2][item] ** 2
    try:
        return prod / sqrt(denom1 * denom2)
    except ZeroDivisionError:
        return 0

In [15]:
def map_func(i):
    return [cosine_similarity(data, i, other) for other in item_idsss]


def get_sim(n, item_ids, parallel=False, symmetric=True, n_jobs=4):
    if not parallel and not symmetric:
        print("not symmetric")
        sim = np.array([[cosine_similarity(data, i, other) for other in item_ids] for i in item_ids])
    if not parallel and symmetric:
        print("symetric")
        sim = np.zeros((n, n))
        for i in item_ids:   ################# item_ids
            sim[i, i] = 1.0
            for j in item_ids[i + 1:]:
                sim[i, j] = cosine_similarity(data, i, j)
                sim[j, i] = sim[i, j]
    if parallel:
        print("multiprocessing")
        with Pool(processes=n_jobs) as p:
            sim = p.map(map_func, item_ids)
            sim = np.array(sim)
    return sim

In [16]:
item_idsss = list(item2id.values())   # 排除 test data
n_items = len(item2id)
%time sim_matrix = get_sim(n_items, item_idsss)

symetric
CPU times: user 23.1 s, sys: 55.9 ms, total: 23.2 s
Wall time: 23.3 s


In [17]:
sim_matrix.shape

(3294, 3294)

In [18]:
sim_whole = {}
for i in range(n_items):
    sim_whole[i] = np.argsort(sim_matrix[i])[::-1]

In [19]:
%%time
k = 20
intersect_user_item = {}
for user, item, _ in ratings(data):
    u_items = list(data[user].keys())
    sim_items = sim_whole[item][:k]
    intersect_items, index_u, _ = np.intersect1d(
        u_items, sim_items, assume_unique=True, return_indices=True)
    intersect_user_item[(user, item)] = (intersect_items, index_u)

CPU times: user 1.76 s, sys: 40 ms, total: 1.8 s
Wall time: 1.8 s


In [22]:
intersect_user_item_test = {}
for user, item in zip(test_user_indices, test_item_indices):
    u_items = list(data[user].keys())
    sim_items = sim_whole[item][:k]
    intersect_items, index_u, _ = np.intersect1d(u_items, sim_items, assume_unique=True, return_indices=True)
    intersect_user_item_test[(user, item)] = (intersect_items, index_u)

In [19]:
def compute_rmse_neighbor_k(k, mode="train"):
    if mode == "train":
        user_indices = train_user_indices
        item_indices = train_item_indices
        ratings = train_ratings
        intersect = intersect_user_item
    elif mode == "test":
        user_indices = test_user_indices
        item_indices = test_item_indices
        ratings = test_ratings
        intersect = intersect_user_item_test
        
    global_mean = np.mean(train_ratings)
    pred_whole = []
    for u, i, r in zip(user_indices, item_indices, ratings):
        intersect_items, index_u = intersect[(u, i)]
        
        if len(intersect_items) == 0:
            pred = global_mean + bu[u] + bi[i]
        else:
            u_ratings = np.array(list(data[u].values()))[index_u]
            base_neighbor = global_mean + bbu[u] + bbi[intersect_items]
            user_sqrt = np.sqrt(len(intersect_items))
            ru = np.sum((u_ratings - base_neighbor) * w[i][intersect_items]) / user_sqrt
            nu = np.sum(c[i][intersect_items]) / user_sqrt
            pred = global_mean + bu[u] + bi[i] + ru + nu
        pred_whole.append(pred)
        
    if mode == "test":
        print("pred: ", pred_whole[:10])

    score = np.sqrt(np.mean(np.power(np.array(pred_whole) - np.array(ratings), 2)))
    return score

In [30]:
np.random.seed(42)
k = 20   ############################
n_users = len(user2id)
n_items = len(item2id)
global_mean = np.mean(train_ratings)
n_epochs = 200
lr = 0.002
reg = 0.1

bu = np.zeros((n_users,))
bi = np.zeros((n_items,))
w = np.random.normal(size=(n_items, n_items))
c = np.random.normal(size=(n_items, n_items))

for epoch in range(n_epochs):
    t0 = time.time()
    for u, i, r in ratings(data):
        intersect_items, index_u = intersect_user_item[(u, i)]
        
        if len(intersect_items) == 0:
            err = r - (global_mean + bu[u] + bi[i])
            bu[u] += lr * (err - reg * bu[u])
            bi[i] += lr * (err - reg * bi[i])
        else:
            u_ratings = np.array(list(data[u].values()))[index_u]
            base_neighbor = global_mean + bbu[u] + bbi[intersect_items]
            user_sqrt = np.sqrt(len(intersect_items))
            ru = np.sum((u_ratings - base_neighbor) * w[i][intersect_items]) / user_sqrt
            nu = np.sum(c[i][intersect_items]) / user_sqrt
            err = r - (global_mean + bu[u] + bi[i] + ru + nu)

            bu[u] += lr * (err - reg * bu[u])
            bi[i] += lr * (err - reg * bi[i])
            w[i][intersect_items] += lr * (err * (u_ratings - base_neighbor) / user_sqrt - 
                                           reg * w[i][intersect_items])
            c[i][intersect_items] += lr * (err / user_sqrt - reg * c[i][intersect_items])
            
    print("Epoch: ", epoch + 1, "\ttime: ", time.time() - t0)
    
    if epoch % 1 == 0:
        t1 = time.time()
        print("Epoch: ", epoch + 1, "\ttraining RMSE: ", compute_rmse_neighbor_k(k, "train"), 
              "\ttest RMSE: ", compute_rmse_neighbor_k(k, "test"))
        print("evaluate time: ", time.time() - t1)

Epoch:  1 	time:  44.69567847251892
Epoch:  1 	training RMSE:  1.2458042553334014 	test RMSE:  1.3722556840047824
evaluate time:  44.410773277282715
Epoch:  2 	time:  44.42795276641846
Epoch:  2 	training RMSE:  1.136692451432997 	test RMSE:  1.3093769475704733
evaluate time:  46.26808190345764
Epoch:  3 	time:  44.11362338066101
Epoch:  3 	training RMSE:  1.0742567200185442 	test RMSE:  1.2669935862011854
evaluate time:  44.48421502113342
Epoch:  4 	time:  43.84966540336609
Epoch:  4 	training RMSE:  1.030746873151682 	test RMSE:  1.2349643020989318
evaluate time:  45.2990300655365
Epoch:  5 	time:  43.515668869018555
Epoch:  5 	training RMSE:  0.9975546886559868 	test RMSE:  1.209428710447678
evaluate time:  45.63119339942932
Epoch:  6 	time:  43.80353546142578
Epoch:  6 	training RMSE:  0.970898049824173 	test RMSE:  1.1883870111829498
evaluate time:  45.02794551849365
Epoch:  7 	time:  44.86887574195862
Epoch:  7 	training RMSE:  0.9487640311135215 	test RMSE:  1.170641436514945
ev

KeyboardInterrupt: 

---

### SuperSVD

In [26]:
list(data.keys()) == list(range(n_users))

False

In [27]:
def compute_rmse_superSVD(k, mode="train"):
    if mode == "train":
        user_indices = train_user_indices
        item_indices = train_item_indices
        ratings = train_ratings
        intersect = intersect_user_item
    elif mode == "test":
        user_indices = test_user_indices
        item_indices = test_item_indices
        ratings = test_ratings
        intersect = intersect_user_item_test
        
    print(len(user_indices), len(item_indices))
    
    nui = np.array([np.sum(yj[list(data[u].keys())], axis=0) / np.sqrt(len(data[u])) for u in range(n_users)])  # data.keys()
    pred1 = global_mean + bu[user_indices] + bi[item_indices] + \
            np.dot(pu + nui, qi.T)[user_indices, item_indices]
   
    
    pred2 = []
    for u, i, r in zip(user_indices, item_indices, ratings):
        intersect_items, index_u = intersect[(u, i)]
        
        if len(intersect_items) == 0:
            pred2.append(0)
        else:
            u_ratings = np.array(list(data[u].values()))[index_u]
            base_neighbor = global_mean + bbu[u] + bbi[intersect_items]
            user_sqrt = np.sqrt(len(intersect_items))
            ru = np.sum((u_ratings - base_neighbor) * w[i][intersect_items]) / user_sqrt
            nu = np.sum(c[i][intersect_items]) / user_sqrt
            pred2.append(ru + nu)
    '''
    if mode == "train":
        print("train pred1: ", pred1[:10])
        print("train pred2: ", pred2[:10])
        print()
    if mode == "test":
        print("test pred1: ", pred1[:10])
        print("test pred2: ", pred2[:10])
        print()
    '''
    pred_whole = pred1 + pred2
    score = np.sqrt(np.mean(np.power(pred_whole - ratings, 2)))
    return score

In [29]:
np.random.seed(42)
k = 20   ############################
n_users = len(user2id)
n_items = len(item2id)
n_factors = 100
global_mean = np.mean(train_ratings)
n_epochs = 200
lr = 0.002
reg = 0.1

bu = np.zeros((n_users,))
bi = np.zeros((n_items,))
pu = np.random.normal(size=(n_users, n_factors))
qi = np.random.normal(size=(n_items, n_factors))
yj = np.random.normal(size=(n_items, n_factors))
w = np.random.normal(size=(n_items, n_items))
c = np.random.normal(size=(n_items, n_items))

for epoch in range(n_epochs):
    t0 = time.time()
    for u, i, r in ratings(data):
        u_items = list(data[u].keys())
        nu_sqrt = np.sqrt(len(u_items))
        nui = np.sum(yj[u_items], axis=0) / nu_sqrt
        dot = np.dot(qi[i], pu[u] + nui)
        intersect_items, index_u = intersect_user_item[(u, i)]
        
        if len(intersect_items) == 0:
            err = r - (global_mean + bu[u] + bi[i] + dot)
            bu[u] += lr * (err - reg * bu[u])
            bi[i] += lr * (err - reg * bi[i])
            qi[i] += lr * (err * (pu[u] + nui) - reg * qi[i])
            pu[u] += lr * (err * qi[i] - reg * pu[u])
            yj[u_items] += lr * (err * qi[u_items] / nu_sqrt - reg * yj[u_items])
        else:
            u_ratings = np.array(list(data[u].values()))[index_u]
            base_neighbor = global_mean + bbu[u] + bbi[intersect_items]
            user_sqrt = np.sqrt(len(intersect_items))
            ru = np.sum((u_ratings - base_neighbor) * w[i][intersect_items]) / user_sqrt 
            nu = np.sum(c[i][intersect_items]) / user_sqrt
            err = r - (global_mean + bu[u] + bi[i] + dot + ru + nu)

            bu[u] += lr * (err - reg * bu[u])
            bi[i] += lr * (err - reg * bi[i])
            qi[i] += lr * (err * (pu[u] + nui) - reg * qi[i])
            pu[u] += lr * (err * qi[i] - reg * pu[u])
            yj[u_items] += lr * (err * qi[u_items] / nu_sqrt - reg * yj[u_items])
            w[i][intersect_items] += lr * (err * (u_ratings - base_neighbor) / user_sqrt - 
                                           reg * w[i][intersect_items])
            c[i][intersect_items] += lr * (err / user_sqrt - reg * c[i][intersect_items])
            
    print("Epoch: ", epoch + 1, "\ttime: ", time.time() - t0)
    
    if epoch % 1 == 0:
        t1 = time.time()
        print("Epoch: ", epoch + 1, "\ttraining RMSE: ", compute_rmse_superSVD(k, "train"), 
              "\ttest RMSE: ", compute_rmse_superSVD(k, "test"))
        print("evaluate time: ", time.time() - t1)

Epoch:  1 	time:  9.48443078994751
75000 75000
24851 24851
Epoch:  1 	training RMSE:  5.3533269684648435 	test RMSE:  10.373572104829243
evaluate time:  1.5428125858306885
Epoch:  2 	time:  10.6420259475708
75000 75000
24851 24851
Epoch:  2 	training RMSE:  3.22334835841845 	test RMSE:  9.147310134110043
evaluate time:  1.4569406509399414
Epoch:  3 	time:  11.131606817245483
75000 75000
24851 24851
Epoch:  3 	training RMSE:  2.3276268807081077 	test RMSE:  8.481751440412177
evaluate time:  1.5579004287719727
Epoch:  4 	time:  9.593751668930054
75000 75000
24851 24851
Epoch:  4 	training RMSE:  1.8446327751310254 	test RMSE:  8.049616719010164
evaluate time:  1.559469223022461
Epoch:  5 	time:  10.412916660308838
75000 75000
24851 24851
Epoch:  5 	training RMSE:  1.534450733787113 	test RMSE:  7.737887271094376
evaluate time:  1.5151734352111816


KeyboardInterrupt: 